In [0]:
scope = 'cxkeyvault'
sas_token_key = 'cxdatalake'
storage_account = 'customeranalysisdatalake'
container3 = 'gold'
container2 = 'silver'

sas_token = dbutils.secrets.get(scope=scope, key=sas_token_key)



import os
from datetime import datetime
from delta.tables import DeltaTable
from pyspark.sql.functions import col, when, sum as _sum, first


In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)


In [0]:
#all paths

accounts_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/accounts/")
customers_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/customers/")
loan_payments_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loan payments/")
loans_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/loans/")
transactions_deltatable_path = (f"abfss://{container2}@{storage_account}.dfs.core.windows.net/transactions/")

total_balance_deltatable_path = (f"abfss://{container3}@{storage_account}.dfs.core.windows.net/total balance/")



In [0]:
# #total balance delta lake empty table

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType


# schema = StructType([ StructField("customer_id", IntegerType(), True),
#                     StructField("first_name", StringType(), True),
#                     StructField("last_name", StringType(), True),
#                     StructField("address", StringType(), True),
#                     StructField("city", StringType(), True),
#                     StructField("state", StringType(), True),
#                     StructField("zip", StringType(), True),
#                     StructField("account_id", IntegerType(), True),
#                     StructField("account_type", StringType(), True),
#                     StructField("total_balance", DecimalType(10,2), True)
#                     ])

# total_balance_deltatable_path = (f"abfss://{container3}@{storage_account}.dfs.core.windows.net/total balance/")

# # Create an empty DataFrame with the schema 
# empty_df = spark.createDataFrame([], schema)

# empty_df.write.format("delta").save(total_balance_deltatable_path)

In [0]:
def get_last_commit_timestamp(deltatable_path):
  # Load the Delta table
  delta_table = DeltaTable.forPath(spark, deltatable_path)

  # Get the history of the Delta table
  history_df = delta_table.history()

  # Get the timestamp of the last commit
  last_commit_timestamp = history_df.orderBy(history_df['version'].desc()).select('timestamp').first()['timestamp']
  return last_commit_timestamp


In [0]:

# Load the Delta tables
accounts_df = spark.read.format("delta").load(accounts_deltatable_path)
customers_df = spark.read.format("delta").load(customers_deltatable_path)
total_balance_df = spark.read.format("delta").load(total_balance_deltatable_path)

# Join the accounts and customers DataFrames on customer_id
joined_df = accounts_df.join(customers_df, "customer_id")

# Group by customer_id and sum the balance
result_df = joined_df.groupBy(
    "customer_id", 
    "first_name", 
    "last_name", 
    "address", 
    "city", #
    "state", 
    "zip"
).agg(
    _sum("balance").alias("total_balance"),
    first("account_id").alias("account_id"),
    first("account_type").alias("account_type")
)

# Select and include all columns
final_df = result_df.select(
    "customer_id",
    "first_name",
    "last_name",
    "address",
    "city",
    "state",
    "zip",
    "account_id",
    "account_type",
    "total_balance"
)



#balance column became decimal(20,2)
final_df = final_df.withColumn("total_balance", col("total_balance").cast("decimal(10,2)"))

# Filter out rows that already exist in total_balance_df
new_data_df = final_df.join(total_balance_df, ["customer_id"], "left_anti")

# Append the new data to the total_balance Delta table
new_data_df.write.format("delta").mode("append").save(total_balance_deltatable_path)

#new_data_df.printSchema()
#spark.read.format("delta").load(total_balance_deltatable_path).printSchema()

# Display the result
if new_data_df.count() > 0:
    display(new_data_df)
else:
    print("no new data to append")



no new data to append
